In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import math
def VisualInspection(ax, f, a, b, dx, **kwargs):
    x = np.arange(a,b+dx,dx)
    y = f(x)
    plt.plot(x, y, **kwargs)
    plt.xlabel('x')
    plt.ylabel('y')

def bracket(f,x1,h):
    c = 1.618033989 
    f1 = f(x1)
    x2 = x1 + h; f2 = f(x2)
  # Determine downhill direction and change sign of h if needed
    if f2 > f1:
        h = -h
        x2 = x1 + h; f2 = f(x2)
      # Check if minimum between x1 - h and x1 + h
        if f2 > f1: return x2,x1 - h 
  # Search loop
    for i in range (100):    
        h = c*h
        x3 = x2 + h; f3 = f(x3)
        if f3 > f2: return x1,x3
        x1 = x2; x2 = x3
        f1 = f2; f2 = f3
    print("Bracket did not find a mimimum")

def search(f,a,b,tol=1.0e-9):
    nIter = int(math.ceil(-2.078087*math.log(tol/abs(b-a))))
    R = 0.618033989
    C = 1.0 - R
  # First telescoping
    x1 = R*a + C*b; x2 = C*a + R*b
    f1 = f(x1); f2 = f(x2)
  # Main loop
    for i in range(nIter):
        if f1 > f2:
            a = x1
            x1 = x2; f1 = f2
            x2 = C*a + R*b; f2 = f(x2)
        else:
            b = x2
            x2 = x1; f2 = f1
            x1 = R*a + C*b; f1 = f(x1)
    if f1 < f2: return x1,f1
    else: return x2,f2

# Example1


### Minimize:
- $\large f(x) = 1.6x^3+3.0 x^2-2.0 x$
- with $\large x \geq = 0$

### Modified problem:
- $\large f^{*}(x) = 1.6x^3+3.0 x^2-2.0 x + \lambda [min(0,x)]^2$


In [ ]:
x = np.arange(0,10,1)
np.minimum(x,0)

In [ ]:
def f(x):
    return 1.6*x**3 + 3.0*x**2 - 2.0*x

def fstar(x):
    lam = 1.0 # Constraint multiplier
    c = np.minimum(0, x) # Constraint function
    return f(x) + lam*c**2

xStart = 1.0
h      = 0.01
x1,x2  = bracket(fstar,xStart,h)
print("bracket", x1, x2)

x,fMin = search(fstar,x1,x2)
print("x =",x)
print("f(x) =",fMin)

fig, ax = plt.subplots(figsize=(12, 8))
VisualInspection(ax, f, -1, 1.5, 0.05, color="blue", linestyle="-", label="f(x)")
VisualInspection(ax, fstar, -1, 1.5, 0.05, color="black", linestyle="-", label="f*(x)")
plt.plot(x, fMin, "o", color="black", markersize=5)
ax.legend()

# Example2

Find R that minimizes the temperature

In [ ]:
q = 50.0
a = 5.0
k = 0.16
h = 20.0
Tinf = 280

def f(r):    
    return (q/2*np.pi)*((np.log(r/a)/k) + (1/(h*r))  ) + Tinf

def fstar(x):
    lam = 100.0 # Constraint multiplier
    c = np.minimum(0, x-a) # Constraint function
    return f(x) + lam*c**2

xStart = a+1.0
h      = 0.01
x1,x2  = bracket(fstar,xStart,h)
print("bracket", x1, x2)

x,fMin = search(fstar,x1,x2)
print("x =",x)
print("f(x) =",fMin)

fig, ax = plt.subplots(figsize=(12, 8))
VisualInspection(ax, f, 3, 40, 0.05, color="blue", linestyle="-", label="f(x)")
VisualInspection(ax, fstar, 3, 40, 0.05, color="black", linestyle="-", label="f*(x)")
plt.plot(x, fMin, "o", color="black", markersize=5)
ax.legend()

# Powell

In [ ]:
def powell(F,x,h=0.1,tol=1.0e-6, sequences=None):
    
    def f(s): return F(x + s*v)    # F in direction of v

    if(sequences is not None):sequences+=[x.copy()]
    n = len(x)                     # Number of design variables
    df = np.zeros(n)               # Decreases of F stored here
    u = np.identity(n)             # Vectors v stored here by rows
    for j in range(30):            # Allow for 30 cycles:
        
        xOld = x.copy()            # Save starting point
        fOld = F(xOld)
      # First n line searches record decreases of F
        for i in range(n):
            v = u[i]
            a,b = bracket(f,0.0,h)
            s,fMin = search(f,a,b)
            df[i] = fOld - fMin
            fOld = fMin
            x = x + s*v
      # Last line search in the cycle    
        v = x - xOld
        a,b = bracket(f,0.0,h)
        s,fLast = search(f,a,b)
        x = x + s*v
        if(sequences is not None):sequences+=[x.copy()]
      # Check for convergence
        if math.sqrt(np.dot(x-xOld,x-xOld)/n) < tol: return x,j+1
      # Identify biggest decrease & update search directions
        iMax = np.argmax(df)
        for i in range(iMax,n-1):
            u[i] = u[i+1]
        u[n-1] = v
    print("Powell did not converge")
        
    

    

# Example3

Find the miminum to
- $\Large 100(y-x^2)^2 + (1-x)^2$

In [ ]:
def F(x): 
    return 100.0*(x[1] - x[0]**2)**2 + (1 - x[0])**2

xStart = np.array([-1.0, 1.0])
sequences = []
xMin,nIter = powell(F,xStart, sequences=sequences)

print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",nIter)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

X = np.arange(-1.5, 1.5, 0.1)
Y = np.arange(-1.5, 1.5, 0.1)
X, Y = np.meshgrid(X, Y)
Z = F(np.array([X,Y]))
ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none')
for i,point in enumerate(sequences):
    z = F(point)
    ax.plot([point[0]],[point[1]],[z], 'o', color="black", markersize=10)
    ax.text(point[0],point[1],z+25, "P%d"%i, color='black',  fontsize=15 )

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(50, 65)


### same with another starting point

In [ ]:
def F(x): 
    return 100.0*(x[1] - x[0]**2)**2 + (1 - x[0])**2

xStart = np.array([-1.0, -1.0])
sequences = []
xMin,nIter = powell(F,xStart, sequences=sequences)

print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",nIter)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

X = np.arange(-1.5, 1.5, 0.1)
Y = np.arange(-1.5, 1.5, 0.1)
X, Y = np.meshgrid(X, Y)
Z = F(np.array([X,Y]))
ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none')
for i,point in enumerate(sequences):
    z = F(point)
    ax.plot([point[0]],[point[1]],[z], 'o', color="black", markersize=10)
    ax.text(point[0],point[1],z+25, "P%d"%i, color='black',  fontsize=15 )

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(50, 65)


## Example 4

- $\Large F(x,y) = (𝑥−5)^2+(𝑦−8)^2$
- With $\Large 𝑥𝑦−5=0 $


In [ ]:
def F(x):    
    return (x[0]-5)**2 + (x[1]-8)**2

def Fstar(x):
    lam = 1.0 #10000.0 # Constraint multiplier
    c = (x[0]*x[1])-5 # Constraint function
    return F(x) + lam*c**2


xStart = np.array([0.0, 0.0])
#xStart = np.array([0.73306759, 7.58776401])
sequences = []
xMin,nIter = powell(Fstar,xStart, sequences=sequences)

print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",nIter)

In [ ]:
#SHOW FIGURE OF F

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

#show the surface
X = np.arange(-10, 10, 0.1)
Y = np.arange(-10, 10, 0.1)
X, Y = np.meshgrid(X, Y)
Z = F(np.array([X,Y]))
ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none')

#show the constraint on the surface
YC = np.arange(0, 10, 1.0)
XC = 5/YC
ZC = F(np.array([XC,YC]))
ax.plot(XC,YC,ZC,color="blue", linewidth=5.0, label="constraint")


for i,point in enumerate(sequences):
    z = F(point)
    ax.plot([point[0]],[point[1]],[z], 'o', color="black", markersize=10)
    ax.text(point[0],point[1],z+3, "P%d"%i, color='black',  fontsize=15 )

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(50, 65)
ax.legend()

#SHOW FIGURE OF F star

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

#show the surface
X = np.arange(-10, 10, 0.1)
Y = np.arange(-10, 10, 0.1)
X, Y = np.meshgrid(X, Y)
Z = Fstar(np.array([X,Y]))
ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none')

for i,point in enumerate(sequences):
    z = F(point)
    ax.plot([point[0]],[point[1]],[z], 'o', color="black", markersize=10)
    ax.text(point[0],point[1],z+3, "P%d"%i, color='black',  fontsize=15 )

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(50, 65)
ax.legend()


## Example 5 (CAN dimension optimization)

In [ ]:

def F(x):    
    r = x[0]
    h = x[1]
    return 2*np.pi*(0.015*r**2 + 0.01*r*h)

def volume(x):
    r = x[0]
    h = x[1]    
    return np.pi*h*r**2

def Fstar(x):
    lam = 1.0E-3 #10000.0 # Constraint multiplier
    c = np.minimum(0,volume(x)-330) # Constraint function
    return F(x) + lam*c**2


xStart = np.array([5.0, 10.0])
#xStart = np.array([3.27049441, 9.81148308])
sequences = []
xMin,nIter = powell(Fstar,xStart, sequences=sequences)

print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",nIter)
print("Volume=", volume(xMin))

In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

#show the surface
X = np.arange(2, 5, 0.05)
Y = np.arange(5, 15, 0.05)
X, Y = np.meshgrid(X, Y)
Z = Fstar(np.array([X,Y]))
#ax.plot_surface(X,Y,np.log(Z+0.0001),cmap='rainbow_r', edgecolor='none', cstride=2, rstride=2)
ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none', cstride=2, rstride=2)



for i,point in enumerate(sequences):
    z = F(point)
    ax.plot([point[0]],[point[1]],[z], 'o', color="black", markersize=10)
    ax.text(point[0],point[1],z+0.2, "P%d"%i, color='black',  fontsize=15 )

ax.set_xlabel('r')
ax.set_ylabel('h')
ax.set_zlabel('log(F)')
#ax.set_zscale("log")
ax.view_init(30, 105)
ax.legend()


# Downhill simplex

In [ ]:
## module downhill
''' x = downhill(F,xStart,side=0.1,tol=1.0e-6)
    Downhill simplex method for minimizing the user-supplied
    scalar function F(x) with respect to the vector x.
    xStart = starting vector x.
    side   = side length of the starting simplex (default is 0.1)
'''
import numpy as np
import math

def downhill(F,xStart,side=0.1,tol=1.0e-6, sequences=None):
    n = len(xStart)                 # Number of variables
    x = np.zeros((n+1,n)) 
    f = np.zeros(n+1)
    
  # Generate starting simplex
    x[0] = xStart
    for i in range(1,n+1):
        x[i] = xStart
        x[i,i-1] = xStart[i-1] + side        
  # Compute values of F at the vertices of the simplex     
    for i in range(n+1): f[i] = F(x[i])

        
  # Main loop
    for k in range(500):
        if(sequences is not None):
            sequences+=[x.copy()]
        
      # Find highest and lowest vertices
        iLo = np.argmin(f)
        iHi = np.argmax(f)       
      # Compute the move vector d
        d = (-(n+1)*x[iHi] + np.sum(x,axis=0))/n
      # Check for convergence
        if math.sqrt(np.dot(d,d)/n) < tol: return x[iLo]
        
      # Try reflection
        xNew = x[iHi] + 2.0*d              
        fNew = F(xNew)        
        if fNew <= f[iLo]:        # Accept reflection 
            x[iHi] = xNew
            f[iHi] = fNew
          # Try expanding the reflection
            xNew = x[iHi] + d               
            fNew = F(xNew)
            if fNew <= f[iLo]:    # Accept expansion
                x[iHi] = xNew
                f[iHi] = fNew
        else:
          # Try reflection again
            if fNew <= f[iHi]:    # Accept reflection
                x[iHi] = xNew
                f[iHi] = fNew
            else:
              # Try contraction
                xNew = x[iHi] + 0.5*d
                fNew = F(xNew)
                if fNew <= f[iHi]: # Accept contraction
                    x[iHi] = xNew
                    f[iHi] = fNew
                else:
                  # Use shrinkage
                    for i in range(len(x)):
                        if i != iLo:
                            x[i] = (x[i] - x[iLo])*0.5
                            f[i] = F(x[i])
    print("Too many iterations in downhill")
    return x[iLo]

In [ ]:

def F(x):   
    r = x[0]
    h = x[1]
    return 2*np.pi*(0.015*r**2 + 0.01*r*h)

def volume(x):
    r = x[0]
    h = x[1]    
    return np.pi*h*r**2

def Fstar(x):
    lam = 1.0E-3 #10000.0 # Constraint multiplier
    c = np.minimum(0,volume(x)-330) # Constraint function
    return F(x) + lam*c**2


xStart = np.array([5.0, 10.0])
#xStart = np.array([3.27049441, 9.81148308])
sequences = []
xMin = downhill(Fstar,xStart, sequences=sequences)
sequences = np.array(sequences)

print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",sequences.shape[0])
print("Volume=", volume(xMin))

In [ ]:
sequences.shape

In [ ]:

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)

#show the surface
X = np.arange(2, 6, 0.05)
Y = np.arange(5, 15, 0.05)
X, Y = np.meshgrid(X, Y)
Z = Fstar(np.array([X,Y]))
#ax.plot_surface(X,Y,np.log(Z+0.0001),cmap='rainbow_r', edgecolor='none', cstride=2, rstride=2)
#ax.plot_surface(X,Y,Z,cmap='rainbow_r', edgecolor='none', cstride=2, rstride=2)

#ax.contourf(X, Y, Z, levels=14, linewidths=0.5, colors='k')
#cntr1 = ax1.contourf(xi, yi, zi, levels=14, cmap="RdBu_r")


from IPython.display import display, clear_output
for i in range(len(sequences)):    
    ax.cla() #clear axes    
    ax.set_xlabel('r')
    ax.set_ylabel('h')    
    ax.contourf(X, Y, np.log(Z), 50, cmap="rainbow_r")    
    
    simplex = sequences[i]
    polygon = plt.Polygon(simplex, True, transform=ax.transData._b, linewidth=5, facecolor="none", edgecolor="blue")
    ax.add_artist(polygon)

    ax.text(0.7, 0.95, "Iteration %d"%i, color='black',  fontsize=25, transform=plt.gca().transAxes )    
    ax.text(0.7, 0.90,"x=%8.3f+-%5.3f"% (np.mean(simplex[:,0]),np.std(simplex[:,0])), color='black',  fontsize=25, transform=plt.gca().transAxes )        
    ax.text(0.7, 0.85,"y=%8.3f+-%5.3f"% (np.mean(simplex[:,1]),np.std(simplex[:,1])), color='black',  fontsize=25, transform=plt.gca().transAxes )        
    ax.text(0.7, 0.80,"volume=%8.3f"% (np.mean(volume(np.transpose(simplex)))), color='black',  fontsize=25, transform=plt.gca().transAxes )            
    
    display(fig)    
    clear_output(wait = True)

plt.plot(sequences[-1][0][0],sequences[-1][0][1], "o", markersize=10, color="blue")


#ax.legend()


In [ ]:
sequences[-1][0]